In [4]:
from serial import Serial
from pyubx2 import UBXReader, NMEA_PROTOCOL, UBX_PROTOCOL
from pyubx2 import UBXMessage, SET_LAYER_RAM, TXN_NONE, POLL, GET
from pyubx2 import POLL_LAYER_RAM

In [5]:
usb_port = '/dev/ttyACM0'
baud_rate = 38400
set_layers = SET_LAYER_RAM
poll_layers = POLL_LAYER_RAM
transaction = TXN_NONE

In [6]:
def msg_send(msg):
    with Serial(usb_port, baud_rate, timeout=3) as stream:
        stream.write(msg)
        
def ubx_set(cfgData, layers=set_layers, transaction=transaction, verbose=False):
    msg = UBXMessage.config_set(layers, transaction, cfgData)
    if verbose:
        print(msg)
    else:
        msg_send(msg.serialize())

def ubx_poll(keys, layers=poll_layers,position=0, verbose=False):
    msg = UBXMessage.config_poll(layers, position, keys)
    if verbose:
        print(msg)
    else:
        with Serial(usb_port, baud_rate, timeout=3) as stream:
            stream.write(msg.serialize())
            ubr = UBXReader(stream, protfilter=NMEA_PROTOCOL | UBX_PROTOCOL)
            raw_data, parsed_data = ubr.read()
            return parsed_data

def ubx_reader(id=None, usb_port=usb_port, baud_rate=baud_rate, verbose=False, timeout=10):
    with Serial(usb_port, baud_rate, timeout=3) as stream:
        t = 0
        while True:
            t = t + 1
            if t > timeout:
                return None
            ubr = UBXReader(stream, protfilter=NMEA_PROTOCOL | UBX_PROTOCOL)
            raw_data, parsed_data = ubr.read()
            if parsed_data is not None:
                if (id is None or parsed_data.identity == id) and verbose:
                    print(parsed_data)
                if id is None or parsed_data.identity == id:
                    return parsed_data
                else:
                    continue


In [14]:
FixTypeList = ['no-fix', 'dead-reckoning-only', '2D-fix', '3D-fix', 'GPS + dead-reckoning', 'Time-only-fix']
SvinVldStatusList = ['not valid', 'valid']
SvinActiveStatusList = ['stop', 'in-progress']

def poll_nav_clk():
    msg = UBXMessage(b'\x01', b'\x22', POLL)
    msg_send(msg.serialize())
    msg = ubx_reader(id='NAV-CLOCK', verbose=True)
    return msg

def get_svin_status():
    msg = ubx_reader(id='TIM-SVIN')
    return msg.dur, SvinVldStatusList[msg.valid], SvinActiveStatusList[msg.active]

def get_fixtype(): 
    msg = ubx_reader(id='NAV-PVT')
    return FixTypeList[msg.fixType]

def set_coordinate(lon, lat, alt):
    
    cfgData = [('CFG_TMODE_LON', int(lon*10**7)), ('CFG_TMODE_LON_HP', int(lon*10**9) - int(lon*10**7)*100), 
            ('CFG_TMODE_LAT', int(lat*10**7)), ('CFG_TMODE_LAT_HP', int(lat*10**9) - int(lat*10**7)*100),
            ('CFG_TMODE_HEIGHT', int(alt*100)), ('CFG_TMODE_HEIGHT_HP', int(alt*10**4) - int(alt*100)*100)]
    ubx_set(cfgData)
    
def set_acc(acc):
    cfgData = ['CFG-TMODE-FIXED_POS_ACC', acc*10**4]
    ubx_set(cfgData)

def set_rtcm():
    cfgData = [('CFG_MSGOUT_RTCM_3X_TYPE1005_USB', 1),
               ('CFG_MSGOUT_RTCM_3X_TYPE1077_USB', 1),
               ('CFG_MSGOUT_RTCM_3X_TYPE1087_USB', 1),
               ('CFG_MSGOUT_RTCM_3X_TYPE1097_USB', 1),
               ('CFG_MSGOUT_RTCM_3X_TYPE1127_USB', 1),
               ('CFG_MSGOUT_RTCM_3X_TYPE1230_USB', 1),
               ('CFG_MSGOUT_RTCM_3X_TYPE4072_1_USB', 1)]
    ubx_set(cfgData)

def get_coordindate():
    msg = ubx_reader(id='NAV-PVT')
    lon = msg.lon
    lat = msg.lat
    alt = msg.height/1000
    return lon, lat, alt

def get_acc():
    msg = ubx_reader(id='NAV-PVT')
    hacc = msg.hAcc/1000
    vacc = msg.vAcc/1000
    return hacc, vacc

def set_gnss():
    cfgData = [ ('CFG_SIGNAL_GPS_ENA', 1), 
                ('CFG_SIGNAL_SBAS_ENA', 0),
                ('CFG_SIGNAL_GAL_ENA', 1),
                ('CFG_SIGNAL_BDS_ENA', 1), 
                ('CFG_SIGNAL_QZSS_ENA', 0), 
                ('CFG_SIGNAL_GLO_ENA', 0),
                ('CFG_SIGNAL_NAVIC_ENA', 0)]
    ubx_set(cfgData)

def get_gnss():
    gps_keys = ['CFG_SIGNAL_GPS_ENA',
                'CFG_SIGNAL_GPS_L1CA_ENA', 
                'CFG_SIGNAL_GPS_L2C_ENA',
                'CFG_SIGNAL_GPS_L5_ENA']
    msg = ubx_poll(gps_keys)
    print('GPS Config:\r')
    for key in gps_keys:
        print('  - %s: %d'%(key, getattr(msg,key)))
    
    sbas_keys = ['CFG_SIGNAL_SBAS_ENA',
                 'CFG_SIGNAL_SBAS_L1CA_ENA']
    msg = ubx_poll(sbas_keys)
    print('SBAS Config:\r')
    for key in sbas_keys:
        print('  - %s: %d'%(key, getattr(msg,key)))

    gal_keys = [ "CFG_SIGNAL_GAL_ENA", 
                 "CFG_SIGNAL_GAL_E1_ENA",
                 "CFG_SIGNAL_GAL_E5A_ENA",
                 "CFG_SIGNAL_GAL_E5B_ENA"]
    msg = ubx_poll(gal_keys)
    print('GALILEO Config:\r')
    for key in gal_keys:
        print('  - %s: %d'%(key, getattr(msg,key)))

    bd_keys = [ "CFG_SIGNAL_BDS_ENA",
                "CFG_SIGNAL_BDS_B1_ENA",
                "CFG_SIGNAL_BDS_B1C_ENA",
                "CFG_SIGNAL_BDS_B2_ENA",
                "CFG_SIGNAL_BDS_B2A_ENA"
                ]
    msg = ubx_poll(bd_keys)
    print('Beidou Config:\r')
    for key in bd_keys:
        print('  - %s: %d'%(key, getattr(msg,key)))
    
    glo_keys = ['CFG_SIGNAL_GLO_ENA',
                'CFG_SIGNAL_GLO_L1_ENA',
                'CFG_SIGNAL_GLO_L2_ENA']
    msg = ubx_poll(glo_keys)
    print('GLONASS Config:\r')
    for key in glo_keys:
        print('  - %s: %d'%(key, getattr(msg,key)))
    '''
    imes_keys = [ "CFG_SIGNAL_IMES_ENA", 
                  "CFG_SIGNAL_IMES_L1_ENA"]
    msg = ubx_poll(imes_keys)
    print('IMES Config:\r')
    for key in imes_keys:
        print('  - %s: %d'%(key, getattr(msg,key)))
    '''
    
    navic_keys = [ "CFG_SIGNAL_NAVIC_ENA",
                   "CFG_SIGNAL_NAVIC_L5_ENA"]
    msg = ubx_poll(navic_keys)
    print('NAVIC Config:\r')
    for key in navic_keys:
        print('  - %s: %d'%(key, getattr(msg,key)))

def master_io_msg():
    cfgData = [('CFG_USBINPROT_UBX',  1),
               ('CFG_USBINPROT_NMEA', 1),
               ('CFG_USBOUTPROT_UBX', 0),
               ('CFG_USBOUTPROT_NMEA',0),
               ('CFG_USBOUTPROT_RTCM',1)]
    ubx_set(cfgData)

def slave_io_msg():
    cfgData = [('CFG_USBINPROT_UBX',  1),
               ('CFG_USBINPROT_NMEA', 1),
               ('CFG_USBINPROT_RTCM', 1),
               ('CFG_USBOUTPROT_UBX', 1),
               ('CFG_USBOUTPROT_NMEA',1)]
    ubx_set(cfgData)

def init_gnss():
    set_gnss()
    # USB input: NMEA, UBX 
    keys = [("CFG_USBINPROT_NMEA", 1), ("CFG_USBINPROT_UBX", 1)]
    ubx_set(keys, verbose=True)
    # USB output: UBX
    keys = [("CFG_USBOUTPROT_NMEA", 0), ("CFG_USBOUTPROT_UBX", 1)]
    ubx_set(keys, verbose=False)
    # enable ubx messages
    keys = [("CFG_MSGOUT_UBX_NAV_PVT_USB", 1), 
        ("CFG_MSGOUT_UBX_NAV_SAT_USB", 1), 
        ("CFG_MSGOUT_UBX_NAV_SIG_USB", 1), 
        ("CFG_MSGOUT_UBX_NAV_STATUS_USB", 1),
        ("CFG_MSGOUT_UBX_RXM_RTCM_USB", 1)]
    ubx_set(keys)
    # set the GPS receiver work in survey-in mode
    # survey-in minimum duration is 1200s
    # survey-in position accuracy limit 0.5m
    keys = [("CFG_TMODE_MODE", 1), ("CFG_TMODE_SVIN_MIN_DUR", 1200), ("CFG_TMODE_SVIN_ACC_LIMIT", 5000)]
    ubx_set(keys, verbose=False)
    # enable ubx-pvt message out from usb
    keys = [("CFG_MSGOUT_UBX_NAV_PVT_USB",1), ("CFG_MSGOUT_UBX_TIM_SVIN_USB", 1)]
    ubx_set(keys)

def check_gnss():
    msg = get_fixtype()
    print(msg)
    msg = get_svin_status()
    print(msg)
    msg = get_coordindate()
    print(msg)
    msg = get_acc()
    print(msg)
    get_gnss()

In [142]:
# USB input: NMEA, UBX 
keys = [("CFG_USBINPROT_NMEA", 1), ("CFG_USBINPROT_UBX", 1)]
ubx_set(keys, verbose=True)
# USB output: UBX
keys = [("CFG_USBOUTPROT_NMEA", 0), ("CFG_USBOUTPROT_UBX", 1)]
ubx_set(keys, verbose=False)



<UBX(CFG-VALSET, version=0, ram=1, bbr=0, flash=0, action=0, reserved0=0, CFG_USBINPROT_NMEA=1, CFG_USBINPROT_UBX=1)>


In [6]:
keys=["CFG_USBOUTPROT_NMEA"]
ubx_poll(keys, verbose=True)

<UBX(CFG-VALGET, version=0, layer=0, position=0, keys_01=276299778)>


In [7]:
keys=["CFG_USBOUTPROT_NMEA"]
ubx_poll(keys)

UBXMessage(b'\x06', b'\x8b', 0, payload=b'\x01\x00\x00\x00\x02\x00x\x10\x00')

In [15]:
init_gnss()

<UBX(CFG-VALSET, version=0, ram=1, bbr=0, flash=0, action=0, reserved0=0, CFG_USBINPROT_NMEA=1, CFG_USBINPROT_UBX=1)>


In [20]:
check_gnss()

3D-fix
(865, 'not valid', 'in-progress')
(-122.2570486, 37.8728505, 92.862)
(0.594, 0.93)
GPS Config:
  - CFG_SIGNAL_GPS_ENA: 1
  - CFG_SIGNAL_GPS_L1CA_ENA: 1
  - CFG_SIGNAL_GPS_L2C_ENA: 1
  - CFG_SIGNAL_GPS_L5_ENA: 0
SBAS Config:
  - CFG_SIGNAL_SBAS_ENA: 0
  - CFG_SIGNAL_SBAS_L1CA_ENA: 0
GALILEO Config:
  - CFG_SIGNAL_GAL_ENA: 1
  - CFG_SIGNAL_GAL_E1_ENA: 1
  - CFG_SIGNAL_GAL_E5A_ENA: 0
  - CFG_SIGNAL_GAL_E5B_ENA: 1
Beidou Config:
  - CFG_SIGNAL_BDS_ENA: 1
  - CFG_SIGNAL_BDS_B1_ENA: 1
  - CFG_SIGNAL_BDS_B1C_ENA: 0
  - CFG_SIGNAL_BDS_B2_ENA: 1
  - CFG_SIGNAL_BDS_B2A_ENA: 0
GLONASS Config:
  - CFG_SIGNAL_GLO_ENA: 0
  - CFG_SIGNAL_GLO_L1_ENA: 1
  - CFG_SIGNAL_GLO_L2_ENA: 1
NAVIC Config:
  - CFG_SIGNAL_NAVIC_ENA: 0
  - CFG_SIGNAL_NAVIC_L5_ENA: 0
